In [ ]:
import os
import sys
import gmsh
import numpy as np
import import_ipynb
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from collections import OrderedDict
from scipy.constants import epsilon_0, speed_of_light
from fem_pre_processing import read_mesh
from fem_pos_processing import graph_results
from IPython.display import display, HTML

# Constants

In [4]:
FREQUENCY = 3E8 # Hz
WAVELENGTH = speed_of_light/FREQUENCY
K0 = 2 * np.pi / WAVELENGTH
OMEGA = 2 * np.pi * FREQUENCY

# `apply_physics()`

In [6]:
def apply_physics(mesh_data):
    # Dictionary with all nodes in the mesh
    nodes_data = mesh_data['nodes']    

    # Adicionar as propriedades do materiais ao dicionário da célula
    for cell in mesh_data['cell'].values():
        x1 = nodes_data[cell['conn'][0]]['xg'][0]
        x2 = nodes_data[cell['conn'][1]]['xg'][0]
        xm = (x1 + x2) * 0.5
        
        # Geometric data    
        cell['geo']['centroid'] = xm
        cell['geo']['jacobian'] = (x2 - x1)

        # Material properties
        er = cell['material']['relative_electric_permittivity']
        mur = cell['material']['relative_magnetic_permeability']
        sigma = cell['material']['electric_conductivity']
        erc = er - 1j * sigma / (OMEGA * epsilon_0)
        u_inc = np.exp(-1j * K0 * xm)
        
        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['p(x)'] = 1/mur
        cell['q(x)'] = -K0**2 * erc
        cell['f(x)'] = K0**2 * (erc - 1) * u_inc
        # cell['f(x)'] = -(K0**2 * cell['p(x)'] + cell['q(x)']) * u_inc

        # Inicializa os contornos do domínio
        gamma_abc, gamma_d, gamma_i = {}, {}, {}
        for idx, node in enumerate(cell['conn']):
            bc_type = nodes_data[node]['bc']['type']

            # Verifica o tipo da condição de contorno
            if bc_type == 'ABC':
                gamma_abc[idx] = node
                # Atualiza o valor da condição de contorno para 'gamma_abc'
                nodes_data[node]['bc']['value'] = 1j * K0 * cell['p(x)']
            
            elif bc_type == 'Dirichlet':
                gamma_d[idx] = node
                # Atualiza o valor da condição de contorno para 'gamma_d'
                nodes_data[node]['bc']['value'] = -np.exp(-1j * K0 * nodes_data[node]['xg'][0])
            
            elif bc_type == 'Interface':
                gamma_i[idx] = node

        # Define o contorno do elemento, caso aplicável
        if gamma_abc:
            cell['contour'] = {'type': 'gamma_abc', 'conn_dict': gamma_abc}
        elif gamma_d:
            cell['contour'] = {'type': 'gamma_d', 'conn_dict': gamma_d}
        elif gamma_i:
            cell['contour'] = {'type': 'gamma_i', 'conn_dict': gamma_i}

            
    return mesh_data

# `apply_pml_physics()`

In [ ]:
def apply_pml_physics(PML_DESIGN, mesh_data):
    # Dictionary with all nodes in the mesh
    nodes_data = mesh_data['nodes']    
    
    # Parâmetros do PML
    x0 = PML_DESIGN['x0']       # Interface do PML
    n = PML_DESIGN['n']         # Ordem do PML 
    R = PML_DESIGN['R']         # Coeficiente de reflexão do truncamento
    SIGMA_0X = -np.log(R) / WAVELENGTH / K0
    print(f"Sigma_0X: {SIGMA_0X}")

    # Adicionar as propriedades do materiais ao dicionário da célula
    for cell in mesh_data['cell'].values():
        x1 = nodes_data[cell['conn'][0]]['xg'][0]
        x2 = nodes_data[cell['conn'][1]]['xg'][0]
        xm = (x1 + x2) * 0.5
        
        # Geometric data    
        cell['geo']['centroid'] = xm
        cell['geo']['jacobian'] = (x2 - x1)

        # Material properties
        er = cell['material']['relative_electric_permittivity']
        mur = cell['material']['relative_magnetic_permeability']
        sigma = cell['material']['electric_conductivity']
        erc = er - 1j * sigma / (OMEGA * epsilon_0)
        u_inc = np.exp(-1j * K0 * xm)

        # PML properties
        if abs(xm) < x0:
            Sx = 1
        else:
            Sx = 1 - 1j * SIGMA_0X * (abs(xm) - x0) ** n 
        cell['Sx'] = Sx

        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['p(x)'] = 1/mur * (1/Sx)
        cell['q(x)'] = -K0**2 * erc * Sx
        cell['f(x)'] = K0**2 * (erc - 1) * u_inc * Sx
        # cell['f(x)'] = -(K0**2 * cell['p(x)'] + cell['q(x)']) * u_inc

        # Inicializa os contornos do domínio
        gamma_abc, gamma_d, gamma_i = {}, {}, {}
        for idx, node in enumerate(cell['conn']):
            bc_type = nodes_data[node]['bc']['type']

            # Verifica o tipo da condição de contorno
            if bc_type == 'ABC':
                gamma_abc[idx] = node
                # Atualiza o valor da condição de contorno para 'gamma_abc'
                nodes_data[node]['bc']['value'] = 1j * K0 * cell['p(x)']
            
            elif bc_type == 'Dirichlet':
                gamma_d[idx] = node
                # Atualiza o valor da condição de contorno para 'gamma_d'
                # nodes_data[node]['bc']['value'] = -np.exp(-1j * K0 * nodes_data[node]['xg'][0])
                # Dirichlet homogêneo no truncamento da PML
                nodes_data[node]['bc']['value'] = 0
            
            elif bc_type == 'Interface':
                gamma_i[idx] = node

        # Define o contorno do elemento, caso aplicável
        if gamma_abc:
            cell['contour'] = {'type': 'gamma_abc', 'conn_dict': gamma_abc}
        elif gamma_d:
            cell['contour'] = {'type': 'gamma_d', 'conn_dict': gamma_d}
        elif gamma_i:
            cell['contour'] = {'type': 'gamma_i', 'conn_dict': gamma_i}

            
    return mesh_data

# `create_p1_domain()`

In [1]:
def create_p1_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, INTERFACES, GEOMETRY, auto_save=True, view_mesh=False):
    mesh_data = {}
    # Define os parâmetros de entrada
    _, ElementOrder = FINITE_ELEMENT

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("lossy_dielectric_slab_abc")
    factory = gmsh.model.occ

    # Dimensões do domínio 
    h = GEOMETRY['h']     # Tamanho do elemento finito
    L = GEOMETRY['L']     # Tamanho do domínio computacional 1-D
    d = GEOMETRY['d']     # Espessura da placa dielétrica
    xa, xb = L/2 - d/2, L/2 + d/2

    # Criar domínio computacional omega_c
    left_fs = factory.addLine(
        factory.addPoint(0, 0, 0, h),
        factory.addPoint(xa, 0, 0, h))

    # Criar domínio computacional omega_c
    lossy_slab = factory.addLine(
        factory.addPoint(xa, 0, 0, h),
        factory.addPoint(xb, 0, 0, h))

    # Criar domínio computacional omega_c
    right_fs = factory.addLine(
        factory.addPoint(xb, 0, 0, h),
        factory.addPoint(L, 0, 0, h))

    # Fragmentar todas as regiões para garantir interfaces conformais
    fragmentedObject = [(1, left_fs), (1, lossy_slab), (1, right_fs)]
    outDimTags, _ = factory.fragment(fragmentedObject, fragmentedObject)

    # Sincronizar após o corte do retângulo interno
    factory.synchronize()

    # Obter as fronteiras do domínio (curva, dim=1)
    BoundariesDimTags = gmsh.model.getBoundary(outDimTags, oriented=True)

    # Obter as fronteira da interface (curva, dim=1)
    InterfacesDimTags = gmsh.model.getBoundary([(1, 2)], oriented=True)

    # Adicionar grupos físicos para pontos (Dim=0)
    for i, dots in enumerate([DimTag[1] for DimTag in BoundariesDimTags]):
        gmsh.model.addPhysicalGroup(0, [dots], tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para pontos (Dim=0)
    for i, dots in enumerate([DimTag[1] for DimTag in InterfacesDimTags]):
        gmsh.model.addPhysicalGroup(0, [dots], tag=INTERFACES[i]['tag'], name=INTERFACES[i]['name'])

    # Adicionar grupos físicos dos contornos (Dim=1)    
    for i, contours in enumerate([[1, 3], [2]]):
        gmsh.model.addPhysicalGroup(1, contours, tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(1)
    gmsh.model.mesh.setOrder(ElementOrder)

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/lossy_dielectric_slab.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)

    # Create mesh Structure Data from gmsh
    read_mesh.basic_info(dim=1)
    mesh_data['nodes'] = read_mesh.get_new_nodes_data(BOUNDARY, INTERFACES, dim=0)
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL, dim=1)
    mesh_data = apply_physics(mesh_data)
    
    gmsh.finalize()
    return mesh_data

# `create_p2_domain()`

In [ ]:
def create_p2_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, INTERFACES, GEOMETRY, auto_save=True, view_mesh=False):
    mesh_data = {}
    type, order = FINITE_ELEMENT

    # Define os parâmetros de entrada
    _, order = FINITE_ELEMENT

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("lossy_dielectric_slab_abc")
    factory = gmsh.model.occ

    # Dimensões do domínio 
    h = GEOMETRY['h']     # Tamanho do elemento finito
    L = GEOMETRY['L']     # Tamanho do domínio computacional 1-D
    d = GEOMETRY['d']     # Espessura da placa dielétrica
    xa = L - d            # Interface entre o vácuo e a placa dielétrica  

    # Criar domínio computacional 
    free_space = factory.addLine(
        factory.addPoint(0, 0, 0, h),
        factory.addPoint(xa, 0, 0, h))

    # Criar domínio computacional 
    lossy_slab = factory.addLine(
        factory.addPoint(xa, 0, 0, h),
        factory.addPoint(L, 0, 0, h))

    # Fragmentar todas as regiões para garantir interfaces conformais
    fragmentedObject = [(1, free_space), (1, lossy_slab)]
    CellsDimTags, _ = factory.fragment(fragmentedObject, fragmentedObject)
    CellsTags = [DimTag[1] for DimTag in CellsDimTags]
    print("ElementsDimTags", CellsDimTags)

    # Sincronizar após o corte do retângulo interno
    factory.synchronize()

    # Obter as fronteiras do domínio (curva, dim=1)
    BoundariesDimTags = gmsh.model.getBoundary(CellsDimTags, oriented=True)
    BoundariesTags = [DimTag[1] for DimTag in BoundariesDimTags]
    print("BoundariesDimTags", BoundariesDimTags)

    # Obter as fronteira da interface (curva, dim=1)
    InterfacesDimTags = gmsh.model.getBoundary([(1, 1)], oriented=True)
    InterfacesDimTags = [DimTag for DimTag in InterfacesDimTags
                          if DimTag[1] not in BoundariesTags]
    InterfacesTags = [DimTag[1] for DimTag in InterfacesDimTags]
    print("InterfacesDimTags", InterfacesDimTags)

    # Adicionar grupos físicos para pontos (Dim=0)
    for i, dot in enumerate(BoundariesTags):
        gmsh.model.addPhysicalGroup(0, [dot], tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para pontos (Dim=0)
    for i, dot in enumerate(InterfacesTags):
        gmsh.model.addPhysicalGroup(0, [dot], tag=INTERFACES[i]['tag'], name=INTERFACES[i]['name'])

    # # Adicionar grupos físicos dos contornos (Dim=1)    
    for i, contours in enumerate(CellsTags):
        gmsh.model.addPhysicalGroup(1, [contours], tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(1)
    gmsh.model.mesh.setOrder(order)

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/pec_backed_dielectric_slab_{type}{order}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)

    # Create mesh Structure Data from gmsh
    read_mesh.basic_info(dim=1)
    mesh_data['nodes'] = read_mesh.get_new_nodes_data(BOUNDARY, INTERFACES, dim=0)
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL, dim=1)
    mesh_data = apply_physics(mesh_data)
    
    gmsh.finalize()
    return mesh_data

# `create_p4_domain()`

In [ ]:
def create_p4_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, INTERFACES, GEOMETRY, auto_save=True, view_mesh=False):
    mesh_data = {}
    # Define os parâmetros de entrada
    _, ElementOrder = FINITE_ELEMENT

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("lossy_dielectric_pml")
    factory = gmsh.model.occ

    # Dimensões do domínio 
    h = GEOMETRY['h']     # Tamanho do elemento finito
    L = GEOMETRY['L']     # Tamanho do domínio físico 1-D
    d = GEOMETRY['d']     # Espessura da placa dielétrica
    xa, xb, xc = d/2, L/2, L
    
    # Criar domínio da PML
    left_pml = factory.addLine(
        factory.addPoint(-xc, 0, 0, h),
        factory.addPoint(-xb, 0, 0, h))
    
    # Criar domínio computacional omega_c
    left_fs = factory.addLine(
        factory.addPoint(-xb, 0, 0, h),
        factory.addPoint(-xa, 0, 0, h))

    # Criar domínio computacional omega_c
    lossy_slab = factory.addLine(
        factory.addPoint(-xa, 0, 0, h),
        factory.addPoint(+xa, 0, 0, h))

    # Criar domínio computacional omega_c
    right_fs = factory.addLine(
        factory.addPoint(xa, 0, 0, h),
        factory.addPoint(xb, 0, 0, h))
    
    # Criar domínio da PML
    right_pml = factory.addLine(
        factory.addPoint(xb, 0, 0, h),
        factory.addPoint(xc, 0, 0, h))

    # Fragmentar todas as regiões para garantir interfaces conformais
    fragmentedObject = [(1, left_pml), (1, left_fs), (1, lossy_slab), (1, right_fs), (1, right_pml)]
    CellsDimTags, _ = factory.fragment(fragmentedObject, fragmentedObject)
    CellsTags = [DimTag[1] for DimTag in CellsDimTags]
    print("ElementsDimTags", CellsDimTags)

    # Sincronizar após o corte do retângulo interno
    factory.synchronize()

    # Obter as fronteiras do domínio (curva, dim=1)
    BoundariesDimTags = gmsh.model.getBoundary(CellsDimTags, oriented=True)
    BoundariesTags = [DimTag[1] for DimTag in BoundariesDimTags]
    print("BoundariesDimTags", BoundariesDimTags)

    # Obter as fronteira da interface (curva, dim=1)
    InterfacesDimTags = gmsh.model.getBoundary([(1, 2), (1, 4)], oriented=True)
    InterfacesDimTags = [DimTag for DimTag in InterfacesDimTags
                          if DimTag[1] not in BoundariesTags]
    InterfacesTags = [DimTag[1] for DimTag in InterfacesDimTags]
    print("InterfacesDimTags", InterfacesDimTags)

    # Adicionar grupos físicos para pontos (Dim=0)
    for i, dot in enumerate(BoundariesTags):
        gmsh.model.addPhysicalGroup(0, [dot], tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para pontos (Dim=0)
    PmlInterfacesTags = [InterfacesTags[0], InterfacesTags[-1]]
    SlabInterfacesTags = [InterfacesTags[1], InterfacesTags[-2]]
    for i, dot in enumerate([PmlInterfacesTags, SlabInterfacesTags]):
        gmsh.model.addPhysicalGroup(0, dot, tag=INTERFACES[i]['tag'], name=INTERFACES[i]['name'])

    # Adicionar grupos físicos dos contornos (Dim=1)
    PmlTags = [CellsTags[0], CellsTags[-1]]
    FreeSpaceTags = [CellsTags[1], CellsTags[-2]]
    SlabTags = [CellsTags[2]]
    for i, contours in enumerate([PmlTags, FreeSpaceTags, SlabTags]):
        gmsh.model.addPhysicalGroup(1, contours, tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(1)
    gmsh.model.mesh.setOrder(ElementOrder)

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/lossy_dielectric_pml.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)

    # Create mesh Structure Data from gmsh
    read_mesh.basic_info(dim=1)
    mesh_data['nodes'] = read_mesh.get_new_nodes_data(BOUNDARY, INTERFACES, dim=0)
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL, dim=1)
    # mesh_data = apply_physics(mesh_data)
    
    gmsh.finalize()
    return mesh_data

# `plot_mesh()`

In [ ]:
def plot_mesh(mesh_data, Domain='Entire'):
    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    nodes_data = mesh_data['nodes']
    xg, yg, _ = graph_results.structured_data(mesh_data)

    plt.figure(figsize=(8, 6))
    # Plotando a malha de elementos finitos 1d
    plt.plot(xg, yg, color='gray', linestyle='-', linewidth=1, zorder=1) 

    # Adicionando números dos nós e elementos
    if len(nodes_data) < 25: 
        # Adicionando os números dos nós           
        for key, node in nodes_data.items():
            x, y = node['xg'][0], node['xg'][1]
            plt.scatter(x, y, color='white', edgecolor='black', s=120, zorder=2)
            plt.text(x, y, str(key), color='red', fontsize=6, ha='center', va='center')

        # Adicionando os números dos elementos
        for key, cell in mesh_data['cell'].items():
            x_c = np.mean([nodes_data[node]['xg'][0] for node in cell['conn']])
            y_c = np.mean([nodes_data[node]['xg'][1] for node in cell['conn']])
            plt.text(x_c, y_c, str(key), fontweight='bold',
                        color='black', fontsize=5, ha='center', va='top')
            
    else:
        plt.scatter(xg, yg, color='black', s=1, zorder=3)   
        
    # Ajustando rótulos e layout
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.axis('equal')
    plt.tight_layout()
    
    # Salvando o arquivo no formato SVG
    filepath = graph_results.get_dir(f"pre_processing/pictures/meshed_{Domain}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `plot_coordinates()`

In [12]:
def plot_coordinates(mesh_data, Domain='Entire'):
    plt.figure(figsize=(8, 3))
    # Dicionário para rastrear os rótulos já adicionados
    labels_shown = {}
    
    for node in mesh_data['nodes'].values():
        x, y = node['xg']
        node_type = node['bc']['type']
        
        # Define o rótulo apenas na primeira vez que o tipo aparece
        if node_type == 'Free':
            label = 'Free' if 'Free' not in labels_shown else None
            labels_shown['Free'] = True
            plt.scatter(x, y, color='gray', label=label, marker='o', s=6, zorder=1)
        
        elif node_type == 'ABC':
            label = 'ABC' if 'ABC' not in labels_shown else None
            labels_shown['ABC'] = True
            plt.scatter(x, y, color='blue', label=label, marker='o', s=12, zorder=2)

        elif node_type == 'Interface':
            label = 'Interface' if 'Interface' not in labels_shown else None
            labels_shown['Interface'] = True
            plt.scatter(x, y, color='red', label=label, marker='o', s=12, zorder=3)

        elif node_type == 'Dirichlet':
            label = 'Dirichlet' if 'Dirichlet' not in labels_shown else None
            labels_shown['Dirichlet'] = True
            plt.scatter(x, y, color='black', label=label, marker='x', s=12, zorder=4)
    
    # Linhas auxiliares no gráfico
    plt.axhline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.axvline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.gca().set_aspect('equal', adjustable='box')
    
    # Remover marcadores do eixo y
    plt.yticks([])
    
    # Configurando a legenda fora do gráfico
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))    
    plt.xlabel(r'$x$ (m)')
    plt.tight_layout()  

    # Salvando o arquivo no formato SVG
    filepath = graph_results.get_dir(f"pre_processing/pictures/coordinates_map_{Domain}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `plot_fem_solution()`

In [ ]:
def plot_fem_solution(MATERIAL, mesh_data, uh, Domain='Entire', type='abs'):
    # Materiais do domínio
    sigma = MATERIAL[1]['electric_conductivity']
    er = MATERIAL[1]['relative_electric_permittivity']

    # Verificação do tipo de solução
    SOLUTION_MAP = {'real': np.real, 'imag': np.imag, 'abs': np.abs}
    if type not in SOLUTION_MAP:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")
    
    # Ordenando os nós pela coordenada x (primeiro valor de xg)
    sorted_nodes = sorted(mesh_data['nodes'].items(), key=lambda item: item[1]['xg'][0])
    
    # Inicializando as variáveis
    xg, us, ui, ut = [], [], [], []
    idx_a, idx_b, idx_pec = None, None, None

    # Laço único para processar todas as informações
    for i, (node_id, node) in enumerate(sorted_nodes):
        # Extraindo as coordenadas e soluções ordenadas
        xg.append(node['xg'][0])
        
        # Campo espalhado numérico
        us.append(uh[node_id])
        
        # Onda incidente
        ui.append(np.exp(-1j * K0 * node['xg'][0]))
        
        # Campo total
        ut.append(us[-1] + ui[-1])
        
        # Identificando os índices de xa
        if node['bc']['name'] in ['xa']:
            idx_a = i
        
        # Identificando os índices de xb
        if node['bc']['name'] in ['xb']:
            idx_b = i
        
        if node['bc']['name'] in ['xL'] and idx_b is None:
            idx_b = i

        # Identificando os índices PEC
        if node['bc']['type'] in ['Dirichlet']:
            idx_pec = i

    # Verificando se os índices foram encontrados
    if idx_a is None or idx_b is None:
        raise ValueError("Nós 'xa' ou 'xb' não encontrados no mesh_data.")

    # Criando o gráfico
    plt.figure(figsize=(12, 8))
    plt.plot(xg, SOLUTION_MAP[type](us), 'r-',  label=r'$|E_z^s|$ (Scattered Field)', linewidth=1)
    plt.plot(xg, SOLUTION_MAP[type](ui), 'b--', label=r'$|E_z^i|$ (Incident Field)', linewidth=1)
    plt.plot(xg, SOLUTION_MAP[type](ut), 'g-',  label=r'$|E_z|$ (Total Field)', linewidth=2)

    # Obtenha os limites automáticos do eixo y
    y_min, y_max = plt.gca().get_ylim()

    # Adicionando a região sombreada entre xa e xb
    if Domain in ['DielectricSlab', 'PecBacked']:
        plt.fill_between(xg[idx_a:idx_b+1], y_min, y_max,
                        color='gray', alpha=0.5, label="Lossy Dielectric Slab")
    
    # Adicionando uma linha sobre o domínio PEC
    if idx_pec is not None:
        plt.axvline(xg[idx_pec], color='black', linestyle='-', linewidth=2, label='PEC Boundary')

    # Configurações do gráfico
    plt.xlabel(r'$x$ (m)')
    plt.title(
        'Scattered and total fields for lossy dielectric slab\n'
        fr'$\varepsilon_r = {er}$ and $\sigma = {sigma}$ S/m', 
        loc='center'
    )
    plt.grid(True, linestyle='--', alpha=0.7)
    if type == 'abs':
        plt.ylabel(r'$|E_z| (V/m)$')
    elif type == 'real':
        plt.ylabel(r'$\Re(E_z) (V/m)$')
    else:
        plt.ylabel(r'$\Im(E_z) (V/m)$')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))    
    plt.tight_layout()

    # Salvando o gráfico no formato SVG
    filepath = graph_results.get_dir(f"pos_processing/pictures/fem_solution_{type}_{Domain}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")


# `plot_fem_pml_solution()`

In [ ]:
def plot_fem_pml_solution(MATERIAL, mesh_data, uh, Name, Domain='Computational', type='abs'):    
    # Materiais do domínio
    sigma = MATERIAL['electric_conductivity']
    er = MATERIAL['relative_electric_permittivity']

    # Verificação do tipo de solução
    SOLUTION_MAP = {'real': np.real, 'imag': np.imag, 'abs': np.abs}
    if type not in SOLUTION_MAP:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")

    # Usar OrderedDict para preservar a ordem de inserção
    physical_nodes = OrderedDict()
    nodes_lossy_slab = OrderedDict()
    nodes_left_pml = OrderedDict()
    nodes_right_pml = OrderedDict()

    # Identificando dos nós de cada região
    for cell in mesh_data['cell'].values():
        # Nós da placa dielétrica
        if cell['material']['name'] == 'lossy_slab':
            for node in cell['conn']:
                nodes_lossy_slab[node] = None
        
        # Nós da PML
        if cell['material']['name'] == 'pml':
            if cell['geo']['centroid'] < 0:
                for node in cell['conn']:
                    nodes_left_pml[node] = None
            else:
                for node in cell['conn']:
                    nodes_right_pml[node] = None

        # Nós do domínio físico
        if cell['material']['name'] not in ['pml']:
            for node in cell['conn']:
                physical_nodes[node] = None

    # Converter de volta para listas, se necessário
    physical_nodes = list(physical_nodes.keys())
    nodes_lossy_slab = list(nodes_lossy_slab.keys())
    nodes_left_pml = list(nodes_left_pml.keys())
    nodes_right_pml = list(nodes_right_pml.keys())
            
    # Inicializando as variáveis
    xg, us, ui, ut = [], [], [], []

    # Ordenando os nós pela coordenada x (primeiro valor de xg)
    sorted_nodes = sorted(mesh_data['nodes'].items(), key=lambda item: item[1]['xg'][0])
    for (node_id, node) in sorted_nodes:
        if Domain == 'Computational':
            xg.append(node['xg'][0])                    # Extraindo as coordenadas e soluções ordenadas
            us.append(uh[node_id])                      # Campo espalhado numérico
            ui.append(np.exp(-1j * K0 * node['xg'][0])) # Onda incidente
            ut.append(us[-1] + ui[-1])                  # Campo total
        else:
            if node_id in physical_nodes:
                xg.append(node['xg'][0])
                us.append(uh[node_id])
                ui.append(np.exp(-1j * K0 * node['xg'][0]))
                ut.append(us[-1] + ui[-1])

    # Criando o gráfico
    plt.figure(figsize=(12, 8))
    plt.plot(xg, SOLUTION_MAP[type](us), 'r-',  label=r'$|E_z^s|$ (Scattered Field)', linewidth=1)
    plt.plot(xg, SOLUTION_MAP[type](ui), 'g--', label=r'$|E_z^i|$ (Incident Field)', linewidth=1)
    plt.plot(xg, SOLUTION_MAP[type](ut), 'b-',  label=r'$|E_z^t|$ (Total Field)', linewidth=1)

    # Obtenha os limites automáticos do eixo y
    y_min, y_max = plt.gca().get_ylim()

    # Adicionando a região dielétrica
    plt.fill_between([mesh_data['nodes'][idx]['xg'][0] for idx in nodes_lossy_slab], 
                        y_min, y_max, color='blue', alpha=0.2, label="Lossy Dielectric Slab")   

    if Domain == 'Computational':
        # Adicionando a região da PML esquerda
        plt.fill_between([mesh_data['nodes'][idx]['xg'][0] for idx in nodes_left_pml], 
                            y_min, y_max, color='gray', alpha=0.2, label="PML")

        # Adicionando a região da PML direita
        plt.fill_between([mesh_data['nodes'][idx]['xg'][0] for idx in nodes_right_pml], 
                            y_min, y_max, color='gray', alpha=0.2)

    # Configurações do gráfico
    plt.xlabel(r'$x$ (m)')
    plt.title(
        'Scattered and total fields for lossy dielectric slab\n'
        fr'$\varepsilon_r = {er}$ and $\sigma = {sigma}$ S/m', 
        loc='center'
    )
    plt.grid(True, linestyle='--', alpha=0.7)
    if type == 'abs':
        plt.ylabel(r'$|E_z| (V/m)$')
    elif type == 'real':
        plt.ylabel(r'$\Re(E_z) (V/m)$')
    else:
        plt.ylabel(r'$\Im(E_z) (V/m)$')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))    
    plt.tight_layout()

    # Salvando o gráfico no formato SVG
    filepath = graph_results.get_dir(f"pos_processing/pictures/fem_solution_{type}_{Domain}_{Name}.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

In [ ]:
def table_resume(data_resume, exact_values):
    # Função auxiliar para calcular erro percentual
    def error(value, exact):
        return abs((exact-value) / exact) * 100

    # Configurando a renderização Latex para a coluna de comprimentos
    def render_latex(df):
        html = df.to_html(escape=False)
        display(HTML(html))
    
    # Preparando os dados em formato de lista para o DataFrame
    rows = []
    lengths = ["lambda/20", "lambda/40", "lambda/80"]
    for key, length in zip(data_resume.keys(), lengths):
        R = data_resume[key]['R']
        T = data_resume[key]['T']
        rows.append({
            "Element Length": length,
            "|R|": f"{R:.4f}",  
            "Error in |R| (%)": f"{error(R, exact_values['R']):.2f}",  
            "|T|": f"{T:.4f}",  
            "Error in |T| (%)": f"{error(T, exact_values['T']):.2f}"  
        })

    # Renderizando o DataFrame com Latex
    render_latex(pd.DataFrame(rows))

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.